In [1]:
import os,re,csv,sys,ast,json,time,urllib,random,spotipy,requests,datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# 原始所有歌手的清單 (透過collectArtists.ipnb整理)
df_artists = pd.read_excel(r'csv/20200303/v3/artist_list_v3.xlsx')
df_artists

,Artist
0,Alexis Michelle
1,Alice Vassar LaCour
2,Amber Hurst Martin
3,Andy Love
4,Anna Clendening
...,...
399,Wesley Eure
400,Will Ryan
401,Willie Colón
402,Zavel Kwartin


In [3]:
# spotifyAPI存取權限
# access token要隨時更新 (https://spotify-token-finder.herokuapp.com/)
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
access_token = 'BQDlZbd56qDXld37D1rqe67aRAlch0TWIoyafXB5Y0hTY86xBlblaiO97OxmaJ_JhE8_IzZjOHP51p_eBRM'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# 取得歌手資訊
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    # 歌手有items資訊
    if items:
        data = items[0]
        if len(items) > 0:
            artist_name = items[0]['name']
            artist_id = items[0]['id']
            artist_url = items[0]['external_urls']['spotify']
            artist_info = (artist_name,artist_id,artist_url)
            #print(artist_info)
            return items[0]
        else:
            return None
    # 歌手沒有items資訊，則回傳check值為1
    else:
        print('None items')
        check = 1
        return check
        pass

In [5]:
# 取得歌手名稱和類型
# https://github.com/Leats/spotify-historymaker/blob/master/spotifyhistorymaker.py
# Bruno Mars = 0du5cEVh5yTK9QJze8zA0C / Adele = 4dpARuHxo51G3z768sgnrY
def get_artist_name_genres(access_token: str, artist_id: str):    
    bearer_token = f'Bearer {access_token}'
    track_headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': bearer_token}
    response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=track_headers)
    response.raise_for_status()
    getname = response.json()['name'] # 型態str
    getid = response.json()['id'] # 型態str
    getgenres = response.json()['genres'] # 型態list
    getgenres2 = ','.join(getgenres) # 型態str
    
    list_artist_name = []
    list_artist_id = []
    list_artist_genres = []
    list_artist_name.append(getname)
    list_artist_id.append(getid)
    list_artist_genres.append(getgenres2)
    return list_artist_genres

In [10]:
# 輸出歌手名稱+Id+Genres的csv(artist_list_ID+Genres_版本.csv)
with open("csv/20200303/v3/artist_list_ID+Genres_v3.csv", "a", encoding="utf_8_sig", newline='') as csvfile:
    writer = csv.writer(csvfile)
    #第一次寫入:w，之後寫入:a
    #writer.writerow(['Artist','Id','Genres'])
    df_artists = pd.read_excel(r'csv/20200303/v3/artist_list_v3.xlsx')
    base = 400
    df = df_artists[['Artist']][400:] #base:base+100/最後一個100[400:]
    L_artist_name = []
    L_artist_id = []
    L_artist_genres = []
    no_item = []
    for i in range(len(df)):
        artist_name = df['Artist'][base+i]
        print(str(i) + '|' + str(artist_name))
        # 判斷check值是否為1(代表歌手不存在)
        check = get_artist(artist_name)
        if check != 1:         
            try:
                artist_info = get_artist(artist_name)
                artist_id = artist_info['id']
                #print(artist_id)
                list_artist_genres = get_artist_name_genres(access_token,artist_id)
                artist_genres = ",".join(list_artist_genres) # list >>>str
                L_artist_name.append(artist_name)
                L_artist_id.append(artist_id)
                L_artist_genres.append(list_artist_genres)
            except:
                L_artist_id.append('')
                L_artist_genres.append('')
                pass
            #之後寫入
            writer.writerow([artist_name, artist_id, artist_genres])
        else:
            no_item.append(artist_name)
            pass

0|Will Ryan
1|Willie Colón
2|Zavel Kwartin
3|Zona Maie Griswold
None items


In [11]:
# 要保留的歌手類型清單  (透過getArtistGenres.ipnb整理)
df_genres_select = pd.read_excel(r'csv/20200303/artist_genres_select.xlsx')
df_genres_select.drop('Unnamed: 0',axis=1,inplace=True)
df_genres_select

,Genres
0,pop
1,dance pop
2,post-teen pop
3,urban contemporary
4,indie poptimism
...,...
1725,swedish choir
1726,cathedral choir
1727,atmosphere
1728,wind quintet


In [12]:
# 讀取artist_list_ID+Genres_版本.csv
df_id_genres = pd.read_csv(r'csv/20200303/v3/artist_list_ID+Genres_v3.csv')
df_id_genres

,Artist,Id,Genres
0,Alexis Michelle,73xhcbpxTXPP7YIAQvPt96,NaN
1,Andy Love,4psWzLHJ7pHepkJn6euY1S,NaN
2,Anna Clendening,2y9j8qUwBMGPYPQJJxmYzD,"pop,post-teen pop,social media pop,viral pop"
3,Arnold McCuller,50khxLzwytXnqh7kXhThAs,NaN
4,Arthur Collins,178cYtyyWSWFgPhv3aUMDN,vaudeville
...,...,...,...
305,Wayne Perkins,0oTDbGjQMOIgGTnYGoh4W1,NaN
306,Wazmo Nariz,1OEWhroX3QCiQACq9E6fjU,"chicago punk,zolo"
307,Will Ryan,4uKt0y4zMxjivxkjDsT3gl,"compositional ambient,neo-classical"
308,Willie Colón,7x5Slu7yTE5icZjNsc3OzW,"boogaloo,latin jazz,salsa,tropical"


In [13]:
# 刪除Genres為空的那一列
df_id_genres_drop = df_id_genres.dropna(axis=0, subset=['Genres'])
df_id_genres_drop.reset_index(inplace=True,drop=True)
df_id_genres_drop.to_csv('csv/20200303/v3/artist_list_ID+Genres_drop_v3.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,Anna Clendening,2y9j8qUwBMGPYPQJJxmYzD,"pop,post-teen pop,social media pop,viral pop"
1,Arthur Collins,178cYtyyWSWFgPhv3aUMDN,vaudeville
2,Arto Lindsay,2914JagOufmP5BY9qLDxtF,no wave
3,Baby Rose,6Z4JcgqrqgysyHIPRtDIHo,"alternative r&b,indie r&b"
4,Bill Champlin,5pLqzTJecAKKbWw0MrDASt,"deep soft rock,yacht rock"
...,...,...,...
94,Vance Gilbert,4d1JIoUa0YcXee1Dn3Yzs0,contemporary folk
95,Wazmo Nariz,1OEWhroX3QCiQACq9E6fjU,"chicago punk,zolo"
96,Will Ryan,4uKt0y4zMxjivxkjDsT3gl,"compositional ambient,neo-classical"
97,Willie Colón,7x5Slu7yTE5icZjNsc3OzW,"boogaloo,latin jazz,salsa,tropical"


In [14]:
# 逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count = 0
delete_index = []
df_id_genres_drop = pd.read_csv(r'csv/20200303/v3/artist_list_ID+Genres_drop_v3.csv')
for row in range(len(df_id_genres_drop)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1 = df_genres_select['Genres'].tolist() # 總類別清單
    List2 = df_id_genres_drop['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中 (any:歌手任何一個類別不包含在總類別中就不刪除/all:歌手所有類別都在總類別中就刪除)
    check = all(item in List1 for item in List2)
    
    # 有包含
    if check is True:
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
        set1 = set(List1) 
        set2 = set(List2)
        print(str(count) + ' : ' + 'List1 ⊂ List2' + '\n' + 'Contain Genres:' + str(list(set1.intersection(set2))) + '\n' + '-'*90)
    # 沒包含
    else:
        print(str(count) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index.append(count)
        pass
    count += 1

print('Delete Index:',delete_index)

0 : List1 ⊂ List2
Contain Genres:['social media pop', 'post-teen pop', 'viral pop', 'pop']
------------------------------------------------------------------------------------------
1 : List1 ⊂ List2
Contain Genres:['vaudeville']
------------------------------------------------------------------------------------------
2 : List1 ⊄ List2
------------------------------------------------------------------------------------------
3 : List1 ⊄ List2
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊄ List2
------------------------------------------------------------------------------------------
6 : List1 ⊄ List2
------------------------------------------------------------------------------------------
7 : List1 ⊄ List2
------------------------------------------------------------------------------------------
8 : List1 ⊂ List2
Contain

In [15]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
# https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop.drop(df_id_genres_drop.index[[delete_index]],inplace=True)
df_id_genres_drop.reset_index(drop=True, inplace=True)
df_id_genres_drop.to_csv('csv/20200303/v3/artist_list_ID+Genres_select_v3.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

D:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:4291: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


,Artist,Id,Genres
0,Anna Clendening,2y9j8qUwBMGPYPQJJxmYzD,"pop,post-teen pop,social media pop,viral pop"
1,Arthur Collins,178cYtyyWSWFgPhv3aUMDN,vaudeville
2,Byron G. Harlan,42nBvukwI9UBCnWTyULDHq,vaudeville
3,Crispin Glover,13rOMazf6oo1Vgix6hm9Jn,outsider
4,David Coffin,2FCz6DEIoFeyMd4gm740Nx,shanty
5,Don Edwards,0feWqfqPgKPFPzqDFE4bSf,cowboy western
6,Edward M. Favor,6dJes2mlWk4V4mVqfbUQCh,vaudeville
7,Edward Meeker,2bXgUrF2xgiSYS85RsEv5U,vaudeville
8,Garrett Gardner,2KiFXwdlGY50sNxIDUOVhA,deep talent show
9,Grace McLean,3r29KIdA9ZoETZ5Ix9x4jS,"broadway,show tunes"


# ---

In [53]:
# Test逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count2 = 0
delete_index2 = []
df_id_genres_drop2 = pd.read_csv(r'csv/20200303/v2/artist_list_ID+Genres_drop_v2test.csv')
df_id_genres_drop2

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Abraham Mateo,2bxxlINUlcMQQb39K7IopR,"latin,latin arena pop,latin pop,latin viral po..."
3,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
4,Adam Levine,4bYPcJP5jwMhSivRcqie2n,pop rock
...,...,...,...
337,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
338,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop"
339,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
340,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [54]:
# Test
for row in range(len(df_id_genres_drop2)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1_ = df_genres_select['Genres'].tolist() # 總類別清單
    List2_ = df_id_genres_drop2['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中 (any:歌手任何一個類別不包含在總類別中就不刪除/all:歌手所有類別都在總類別中就刪除)
    check2 = all(item in List1_ for item in List2_)
    
    # 有包含
    if check2 is True:
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
        set1_ = set(List1_) 
        set2_ = set(List2_)
        print(str(count2) + ' : ' + 'List1 ⊂ List2' + '\n' + 'Contain Genres:' + str(list(set1_.intersection(set2_))) + '\n' + '-'*90)
    # 沒包含
    else:
        print(str(count2) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index2.append(count2)
        pass
    count2 += 1

print('Delete Index:',delete_index2)

0 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop', 'pop', 'boy band']
------------------------------------------------------------------------------------------
1 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop']
------------------------------------------------------------------------------------------
2 : List1 ⊄ List2
------------------------------------------------------------------------------------------
3 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop', 'pop', 'idol']
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊄ List2
------------------------------------------------------------------------------------------
6 : List1 ⊄ List2
------------------------------------------------------------------------------------------
7 : List1 ⊂ List2
Contain Genres:['swedish pop', 'europop', 'clas

------------------------------------------------------------------------------------------
309 : List1 ⊄ List2
------------------------------------------------------------------------------------------
310 : List1 ⊄ List2
------------------------------------------------------------------------------------------
311 : List1 ⊂ List2
Contain Genres:['brill building pop']
------------------------------------------------------------------------------------------
312 : List1 ⊄ List2
------------------------------------------------------------------------------------------
313 : List1 ⊄ List2
------------------------------------------------------------------------------------------
314 : List1 ⊄ List2
------------------------------------------------------------------------------------------
315 : List1 ⊂ List2
Contain Genres:['operatic pop']
------------------------------------------------------------------------------------------
316 : List1 ⊄ List2
------------------------------------------

In [55]:
# Test篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
# https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop2.drop(df_id_genres_drop2.index[[delete_index2]],inplace=True)
df_id_genres_drop2.reset_index(drop=True, inplace=True)
df_id_genres_drop2.to_csv('csv/20200303/v2/artist_list_ID+Genres_select_v2test.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop2

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
3,Agnetha Faltskog,7fUtt9kVZOyn9LWy0JbDRI,"classic swedish pop,europop,swedish pop"
4,AJ Mitchell,6dn6x1XOng3LOAnfTjUn77,"pop,post-teen pop"
...,...,...,...
132,Tori Kelly,1vSN1fsvrzpbttOYGsliDr,"dance pop,neo mellow,pop,post-teen pop,viral pop"
133,Vanessa Hudgens,6G9bygHlCyPgNGxK2l3YdE,"dance pop,hollywood,post-teen pop,viral pop"
134,Victoria Beckham,54w1dJI7MTjkRGNfGuu2fc,europop
135,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"


In [154]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
 # https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop.drop(df_id_genres_drop.index[[delete_count]],inplace=True)
df_id_genres_drop.reset_index(drop=True, inplace=True)
df_id_genres_drop.to_csv('csv/20200303/v2/artist_list_ID+Genres_select_v2.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Adam Rickitt,2u1BQaJtganAtbdBrUPZe3,bubblegum dance
3,Agnetha Faltskog,7fUtt9kVZOyn9LWy0JbDRI,"classic swedish pop,classic uk pop,europop,new..."
4,Agnez Mo,6pMR8Zgot664613rAiLC2Z,indonesian pop
...,...,...,...
322,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
323,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."
324,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
325,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [56]:
# 把歌手類型清單的類型欄位轉換成list
# https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_dataframe
List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
#print('List1 =',List1)
print(len(List1))
#print('List2 =',List2)
print(len(List2))

if len(List1) == len(df_genres_select):
    print('Same')

check =  any(item in List1 for item in List2)
if check is True:
    print("List1 ⊂ List2")    
else :
    print("List1 ⊄ List2")

1730
4
Same
The list List1 contains some elements of the list List2


In [60]:
# list取交集的部分
def list_contains(List1, List2):  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        print('Contain Genres:',list(set1.intersection(set2))) # 原始型態set
        return True 
    else: 
        return False

List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
list_contains(List1, List2)

Contain Genres: ['pop', 'post-teen pop', 'boy band', 'dance pop']


True